# Projeto 2 – Parsing web em C++
## Super Computação - 2018/2
### Gabriela Almeida

Para o segundo projeto de super computação trabalhou-se com um problema em que concorrência tem um papel fundamental na obtenção de bom desempenho: download e análise de páginas web. Criou-se um crawler que identifica páginas de produtos em um site de e-commerce e extrai as informações básicas dos produtos. Alguns exemplos são as categorias do site da Magazine Luiza como as seguinte: [DVD Player](https://www.magazineluiza.com.br/dvd-player/tv-e-video/s/et/tvdb/) e [Controle Remoto](https://www.magazineluiza.com.br/controle-remoto/tv-e-video/s/et/cmrt/).

Para demonstra como a concorrência tem um papel fundamental nesse tipo de problema, esse relatório tem como objetivo apontar a diferença de desempenho entre um crawler sequêncial e um paralelo.

## Descrição e Implementação do problema

### Modelo sequencial

A implementação do modelo sequencial foi feita da seguinte forma: dada uma página de exibição de produto, o web crawler extrai as seguintes informações

    1. nome do produto
    2. descrição do produto
    3. url da foto do produto
    4. preço à vista 
    5. preço parcelado
    6. categoria do produto
    7. url da página de axibição

A identificação de páginas de produto é feita a partir de sua categoria, ou seja, o crawler desenvolvido é apontado para uma página com os produtos de uma categoria, como os exemplificados anteriormente, e ele consegue obter as páginas de produto, lindando com possíveis paginação da listagem.

Após a compilação do programa, explicado no README, o programa é executado na linha de comando como:

    ./crawlerSEQ url_da_listagem_por_categoria

A partir dessa url disponibilizada, o programa faz o download do conteúdo html dessa página, usando a biblioteca _curl_: 

Com o html da página de categoria disponível, é possível adquirir as urls dos produtos presentes nela e a da página seguinte a ela. Para fazer isso, usou-se o regex da biblioteca boost. Essa ferramenta se baseia em expressões regulares para identificação de strings.

Sendo possível adquirir todas as urls das páginas de produtos da categoria ,além das url dos produtos dessas páginas, então é possível fazer o download das páginas de todos os produtos e obter as informações desejadas.

O resultado do programa é escrito na saída padrão no formato json. Cada produto é um objeto com os seguintes campos:

    {
        "nome": "",
        "descricao": "",
        "foto": "",
        "preco: 0,
        "preco_parcelado": 0,
        "preco_num_parcelas": 0,
        "categoria": "",
        "url": """
    }

### Modelo paralelo

A implementação do modelo paralelo possui uma lógica parecida com a do sequencial. Assim como o modelo anterior, é feito download da página com os produtos, são identificadas as urls dos produtos presentes nela e a url da página seguinte, é feito o download das paginas dos produtos, extraídas as informações necessárias e após a análise de todos os produtos faz-se o mesmo processo com a página seguinte.

No entanto essa implementação é dada em paralelismo por tarefas, ou seja, é preciso sincronizar tarefas que dependem parcialmente umas das outras (parte de uma tarefa depende de um resultado de outra quanto o restante é independente). Ao exercutar as partes independente de maneira paralela pode-se obter ganhos de desempenho consideráveis em processos longos.

O modelo paralelo por tarefas foi estruturado da seguinte forma:

imagem

É possível perceber no esquema a cima que as threads compartilham de alguns recursos fazendo com que elas sejam interdependentes.

A thread prodLinkCollectorThread é encarregada de fazer o download das páginas com os produtos, identificar as urls dos produtos presentes nelas e colocá-las na listofUrls. Além disso essa thread é encarregada de adquirir a url da pagina com produtos seguinte e fazer o mesmo processo, sucessivamente, até a última página dessa categoria. Desse modo, essa thread terá adicionado à listofUrls as urls de todos os produtos disponíveis naquela categoria.
A segunda thread, prodPageCollectorThread, precisa recolher uma url da listofUrls, fazer o download dela e adicionar-la à listofPages. 
Já prodCollectorThread coleta uma pagina de listofPages, coleta as informações desejadas e as adiciona a uma string global "finalJSON".

Levando em consideração essa estruturação, é possível definir multiplas threads do tipo prodPageCollectorThread fazendo com que elas rodem em paralelo.O mesmo pode acontecer com prodCollectorThread.

imagem

Como essa threads compartilham algumas estruturas, o uso de semáforos é fundamental para coordenar o acesso de cada thread a elas. Desse modo foram definidos os seguintes semáforos:

O primeiro controla o acesso à listofUrls, o segundo o acesso à listofPages e o tercero o acesso à string global "finalJSON". Já os dois últimos valem como contadores de elementos em listofUrls e listofPages respectivamente.

Assim como no sequencial, após a compilação do programa, explicado no README, o programa é executado na linha de comando como:

    ./crawlerPAR url_da_listagem_por_categoria numProducers numConsumers

No qual numProducers é o número de threads do tipo prodPageCollectorThread e numConsumers é o numero de threads do tipo prodCollectorThread.

As threads são definidas da seguinte forma: